In [1]:
import numpy as np
import cv2 # 3.3

In [2]:
cap = cv2.VideoCapture('../../vid/drone-flight.mp4')
ret, frame = cap.read()
print(ret)

True


In [3]:
def run(func, link='../../vid/drone-flight.mp4'):
    cap = cv2.VideoCapture(link)
    while True:
         # Capture frame-by-frame
        ret, frame = cap.read()
        func(ret, frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [4]:
def func(ret, frame):
    cv2.imshow('frame' ,frame)
run(func)

In [5]:
def scale(frame, factor=.5):
    height, wid = frame.shape[:2]
    scaled_wid, scaled_height = np.int(wid * factor), np.int(height * factor)
    frame = cv2.resize(frame, (scaled_wid, scaled_height))
    return frame

In [6]:
def orb_func(ret, frame):
    frame = scale(frame)
    orb = cv2.ORB_create()
    kp = orb.detect(frame)
    # kp, des = orb.compute(frame, kp)
    kp_drawn = cv2.drawKeypoints(frame, kp, None, color=(0, 255, 0))
    cv2.imshow('kp_drawn', kp_drawn)
run(orb_func)

In [7]:
cv2.__version__

'3.3.0'

In [8]:
# try Kmeans

In [9]:
def get_orb_kp(frame, orb=cv2.ORB_create()):
    frame = scale(frame, .5)
    kp = orb.detect(frame)
    kp_drawn = cv2.drawKeypoints(frame, kp, None, color=(0, 255, 0))
    return kp, kp_drawn

In [10]:
def kp_to_pairs(kp, dim):
    max_y, max_x, _ = dim
    out = np.zeros((len(kp), 2))
    for i in range(len(kp)):
        x, y = kp[i].pt
        
        out[i] = np.array([x, max_y-y])
    return out

In [11]:
def get_centers(Z, num=10):
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center=cv2.kmeans(Z,num,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    return center

In [12]:
def plot_boxes(frame, name, centers, dim):
    y_max, x_max, _  = dim
    
    for i in range(len(centers)):
        x, y = centers[i]
        x, y = np.int(x), np.int(y_max - y)
        half_size = 50
        top_left = x - half_size, y - half_size
        bot_right = x + half_size, y + half_size
        frame = cv2.rectangle(frame, top_left, bot_right, (0, 255, 0),  1)
        # frame = cv2.circle(frame, (x, y), 50, (0, 255, 0),  1)
    cv2.imshow(name, frame)

In [13]:
def orb_func_boxed(ret, frame):
    kp, frame = get_orb_kp(frame, orb=cv2.ORB_create(nfeatures=1000))
    dim = frame.shape
    Z = kp_to_pairs(kp, dim)
    centers = get_centers(Z, num=10)
    plot_boxes(frame, 'frame', centers, dim)
    
run(orb_func_boxed)

In [22]:
link='../../vid/drone-flight.mp4'
cap = cv2.VideoCapture(link)
prev_ret, prev_frame = cap.read()
prev_frame = scale(prev_frame, factor=.25)

orb = cv2.ORB_create(nfeatures=1000)
prev_kp, prev_des = orb.detectAndCompute(prev_frame, None)

while True:
    ret, frame = cap.read()
    for i in range(10):
        ret, frame = cap.read()
    frame = scale(frame, factor=.25)
    kp, des = orb.detectAndCompute(frame, None)
    
    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    # Match descriptors.
    matches = bf.match(prev_des, des)
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    # Draw first 10 matches.
    match_frame = cv2.drawMatches(prev_frame, prev_kp, frame, kp, matches[:200], None, flags=2)
    cv2.imshow('match_frame', match_frame)
    
    prev_ret, prev_frame = ret, frame
    prev_kp, prev_des = kp, des    
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [15]:
link= '../../vid/drone-flight.mp4'
cap = cv2.VideoCapture(link)
prev_ret, prev_frame = cap.read()

orb = cv2.ORB_create()
prev_kp, prev_des = orb.detectAndCompute(prev_frame, None)

cv2.imshow('prev_frame', prev_frame)
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

In [19]:
main_frame =  cv2.imread('../../img/feature_mapping_1.jpg')
# main_frame = scale(main_frame, factor=.25)

orb = cv2.ORB_create()
main_kp, main_des = orb.detectAndCompute(main_frame, None)

cap = cv2.VideoCapture('../../vid/drone-flight.mp4')
while True:
    ret, frame = cap.read()
    frame = scale(frame, factor=.5)
    kp, des = orb.detectAndCompute(frame, None)
    
    # create BFMatcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    # Match descriptors.
    matches = bf.match(main_des, des)
    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    # Draw first 10 matches.
    match_frame = cv2.drawMatches(main_frame, main_kp, frame, kp, matches[:10], None, flags=2)
    cv2.imshow('match_frame', match_frame)   
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [21]:
cap = cv2.VideoCapture('../../vid/drone-flight.mp4')
ret, frame = cap.read()
frame = scale(frame, factor=.5)
frame = frame[200:350, 350:500]

frame = scale(frame, factor=.5)
kp, des = orb.detectAndCompute(frame, None)

# create BFMatcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# Match descriptors.
matches = bf.match(main_des, des)
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)
# Draw first 10 matches.
match_frame = cv2.drawMatches(main_frame, main_kp, frame, kp, matches[:10], None, flags=2)
cv2.imshow('match_frame', match_frame) 

cv2.imshow('frame', frame)
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

In [35]:
frame.shape

(540, 960, 3)

In [6]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

MIN_MATCH_COUNT = 10
img1 = cv2.imread('../../img/feature_mapping_1.jpg')

cap = cv2.VideoCapture('../../vid/drone-flight.mp4')
ret, frame = cap.read()
frame = scale(frame, factor=.5)
img2 = cv2.imread('../../img/original/img2.jpg')
# Initiate SIFT detector
sift = cv2.xfeatures2d.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1,des2,k=2)
# store all the good matches as per Lowe's ratio test.
good = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        good.append(m)

        
if len(good)>MIN_MATCH_COUNT:
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    matchesMask = mask.ravel().tolist()
    h,w,d = img1.shape
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,M)
    img2 = cv2.polylines(img2,[np.int32(dst)],True,255,3, cv2.LINE_AA)
else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
    matchesMask = None

    
draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
cv2.imshow('frame', img3)
cv2.waitKey(0)
cap.release()
cv2.destroyAllWindows()

Not enough matches are found - 1/10


Not enough matches are found - 2/10
